# LlamaStack Workshop - Client Demo

This notebook demonstrates how to interact with your deployed LlamaStack distribution programmatically.

## Prerequisites
- You have deployed a model (Llama 3.2-3B) in your project
- You have enabled the GenAI Playground (which creates a LlamaStack Distribution)

## What You'll Learn
1. How to list available models
2. How to list available tools (MCP servers)
3. How to make chat completions
4. How to use tool calling

In [ ]:
# Install required packages
%pip install -q requests

In [ ]:
import requests
import json
import os

# Configuration - UPDATE THIS with your project name!
PROJECT_NAME = "user-XX"  # <-- Change XX to your user number (e.g., user-05)

# LlamaStack endpoint (internal OpenShift service)
LLAMASTACK_URL = f"http://lsd-genai-playground-service.{PROJECT_NAME}.svc.cluster.local:8321"

print(f"Project: {PROJECT_NAME}")
print(f"LlamaStack URL: {LLAMASTACK_URL}")

## 1. List Available Models

Let's see what models are available in your LlamaStack distribution.

In [ ]:
response = requests.get(f"{LLAMASTACK_URL}/v1/models", timeout=10)
models = response.json().get("data", [])

# Filter to LLM models only
llm_models = [m for m in models if m.get("model_type") == "llm"]

print(f"🤖 LLM Models Available: {len(llm_models)}")
print("=" * 50)
for m in llm_models:
    print(f"  • {m.get('identifier')} ({m.get('provider_id')})")
    
# Store the first model ID for later use
# Note: identifier already includes the provider prefix (e.g., "vllm-inference-1/llama-32-3b-instruct")
if llm_models:
    MODEL_ID = llm_models[0].get('identifier')
    print(f"\n📌 Using model: {MODEL_ID}")

## 2. List Available Tools (MCP Servers)

Tools are provided by MCP servers. Let's see what's available.

In [ ]:
response = requests.get(f"{LLAMASTACK_URL}/v1/tools", timeout=10)
data = response.json()
tools = data if isinstance(data, list) else data.get("data", [])

# Group by toolgroup (MCP server)
toolgroups = {}
for t in tools:
    tg = t.get("toolgroup_id", "unknown")
    if tg not in toolgroups:
        toolgroups[tg] = []
    toolgroups[tg].append(t.get("name", "unknown"))

# Count MCP servers (exclude builtin)
mcp_servers = [tg for tg in toolgroups.keys() if tg.startswith("mcp::")]

# Store toolgroup IDs for later use
TOOLGROUPS = [tg for tg in toolgroups.keys() if tg.startswith("mcp::")]

print(f"🛠️ MCP Servers: {len(mcp_servers)}")
print(f"📊 Total Tools: {len(tools)}")
print("=" * 50)
for tg, tool_list in sorted(toolgroups.items()):
    icon = "🌤️" if "weather" in tg else "🔧"
    print(f"\n{icon} {tg} ({len(tool_list)} tools)")
    for tool in tool_list:
        print(f"   • {tool}")
        
print(f"\n📌 Toolgroups for agent: {TOOLGROUPS}")

## 3. Simple Chat Completion

Let's test a basic chat completion.

In [ ]:
payload = {
    "model": MODEL_ID,
    "messages": [
        {"role": "user", "content": "What is the capital of France? Answer in one sentence."}
    ],
    "temperature": 0.7,
    "max_tokens": 256
}

print(f"🤖 Using model: {MODEL_ID}")
print("=" * 50)

response = requests.post(
    f"{LLAMASTACK_URL}/v1/openai/v1/chat/completions",
    json=payload,
    timeout=60
)

if response.status_code == 200:
    result = response.json()
    content = result.get("choices", [{}])[0].get("message", {}).get("content", "")
    print(f"\n📝 Response:")
    print(content)
else:
    print(f"❌ Error: {response.status_code} - {response.text}")

## 4. Test Weather MCP Tool 🌤️

Now let's test the Weather MCP tool using the LlamaStack Agents API, which properly handles tool calling.

In [ ]:
# Step 1: Create an agent with Weather MCP tools
agent_config = {
    "agent_config": {
        "model": MODEL_ID,
        "instructions": "You are a helpful assistant. Use the available tools to answer questions.",
        "toolgroups": TOOLGROUPS,  # This enables the MCP tools!
        "enable_session_persistence": False
    }
}

response = requests.post(f"{LLAMASTACK_URL}/v1/agents", json=agent_config, timeout=30)
if response.status_code == 200:
    agent_id = response.json().get("agent_id")
    print(f"✅ Agent created: {agent_id}")
else:
    print(f"❌ Error creating agent: {response.status_code} - {response.text}")
    agent_id = None

## Step 2: Create a session and ask a question

In [ ]:
# Step 2: Create a session
if agent_id:
    session_response = requests.post(
        f"{LLAMASTACK_URL}/v1/agents/{agent_id}/session",
        json={"session_name": "workshop-session"},
        timeout=30
    )
    if session_response.status_code == 200:
        session_id = session_response.json().get("session_id")
        print(f"✅ Session created: {session_id}")
    else:
        print(f"❌ Error creating session: {session_response.status_code}")
        session_id = None

## Step 3: Ask a question using the Weather MCP tool

# Step 3: Ask a question - the agent will use the Weather MCP tool!
weather_question = "List all available weather stations"

if agent_id and session_id:
    print(f"🌤️ Weather Query: {weather_question}")
    print("=" * 50)
    
    turn_request = {
        "messages": [{"role": "user", "content": weather_question}],
        "stream": False
    }
    
    response = requests.post(
        f"{LLAMASTACK_URL}/v1/agents/{agent_id}/session/{session_id}/turn",
        json=turn_request,
        timeout=120
    )
    
    if response.status_code == 200:
        result = response.json()
        # Extract the assistant's response
        for event in result.get("events", []):
            if event.get("event_type") == "turn_complete":
                turn = event.get("turn", {})
                for msg in turn.get("output_message", {}).get("content", []):
                    if msg.get("type") == "text":
                        print(f"\n📝 Response:")
                        print(msg.get("text", ""))
                # Show tool calls if any
                for step in turn.get("steps", []):
                    if step.get("step_type") == "tool_execution":
                        print(f"\n🔧 Tool called: {step.get('tool_calls', [])}")
    else:
        print(f"❌ Error: {response.status_code} - {response.text}")

## 5. Explore on Your Own!

Try modifying the query below to test different questions. The agent will use the appropriate MCP tools.

**Weather questions:**
- "Get weather statistics"
- "Search for weather in Tokyo"
- "Get current weather for station VIDP"

**HR questions (after Part 3):**
- "List all employees"
- "Get vacation balance for employee EMP001"
- "List all job openings"

Station codes: VIDP = New Delhi, RJTT = Tokyo, KJFK = New York, EGLL = London, YSSY = Sydney

In [ ]:
# Your turn! Modify this query to try different questions
my_question = "Get weather statistics"  # Try changing this!

if agent_id and session_id:
    print(f"🔍 Query: {my_question}")
    print("=" * 50)
    
    turn_request = {
        "messages": [{"role": "user", "content": my_question}],
        "stream": False
    }
    
    response = requests.post(
        f"{LLAMASTACK_URL}/v1/agents/{agent_id}/session/{session_id}/turn",
        json=turn_request,
        timeout=120
    )
    
    if response.status_code == 200:
        result = response.json()
        for event in result.get("events", []):
            if event.get("event_type") == "turn_complete":
                turn = event.get("turn", {})
                for msg in turn.get("output_message", {}).get("content", []):
                    if msg.get("type") == "text":
                        print(f"\n📝 Response:")
                        print(msg.get("text", ""))
    else:
        print(f"❌ Error: {response.status_code} - {response.text}")

---

## 🎯 Part 3: After Adding HR MCP

**Run this section AFTER you complete Part 3 of the workshop** (adding HR MCP to your LlamaStack config).

First, restart the kernel and run all cells again to pick up the new HR tools. Then run the cell below:

In [ ]:
# Test using BOTH MCP servers in one query!
combined_question = "List all weather stations and list all employees"

if agent_id and session_id:
    print(f"🌤️👥 Combined Query: {combined_question}")
    print("=" * 50)
    
    turn_request = {
        "messages": [{"role": "user", "content": combined_question}],
        "stream": False
    }
    
    response = requests.post(
        f"{LLAMASTACK_URL}/v1/agents/{agent_id}/session/{session_id}/turn",
        json=turn_request,
        timeout=180
    )
    
    if response.status_code == 200:
        result = response.json()
        for event in result.get("events", []):
            if event.get("event_type") == "turn_complete":
                turn = event.get("turn", {})
                for msg in turn.get("output_message", {}).get("content", []):
                    if msg.get("type") == "text":
                        print(f"\n📝 Response:")
                        print(msg.get("text", ""))
                # Check if tools were used
                tool_steps = [s for s in turn.get("steps", []) if s.get("step_type") == "tool_execution"]
                if tool_steps:
                    print("\n" + "=" * 50)
                    print("🎉 SUCCESS! Your AI used MCP tools!")
                    for step in tool_steps:
                        for tc in step.get("tool_calls", []):
                            print(f"   🔧 Called: {tc.get('tool_name')}")
    else:
        print(f"❌ Error: {response.status_code} - {response.text}")

## 🎓 Workshop Complete!

Congratulations! You've learned how to:
- ✅ List available models in LlamaStack
- ✅ List available tools (MCP servers)
- ✅ Make chat completions
- ✅ Use Weather MCP tools
- ✅ Use HR MCP tools
- ✅ Combine multiple MCP tools in one query

**Key Takeaway:** You added new capabilities to your AI by just updating a configuration file - no coding required!